In [2]:
%pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 19.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [5]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu




Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class DoodleDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data_frame.iloc[idx]['image_path'])
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64, 64))  # Resize to 64x64 for consistency

        if self.transform:
            image = self.transform(image)

        return image

# Set up data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create dataset and split into train/val
dataset = DoodleDataset(csv_file='/kaggle/input/doodle-dataset/master_doodle_dataframe.csv', 
                        root_dir='/kaggle/input/doodle-dataset', 
                        transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

ModuleNotFoundError: No module named 'torch._custom_ops'

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class SymmetryNet(nn.Module):
    def __init__(self):
        super(SymmetryNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 4)  # 4 outputs: vertical and horizontal symmetry coordinates
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 128 * 8 * 8)
        x = self.fc(x)
        return x

# Initialize models
autoencoder = Autoencoder()
symmetry_net = SymmetryNet()

In [ ]:
# Autoencoder loss and optimizer
autoencoder_criterion = nn.MSELoss()
autoencoder_optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# SymmetryNet loss and optimizer
symmetry_criterion = nn.MSELoss()
symmetry_optimizer = optim.Adam(symmetry_net.parameters(), lr=0.001)

In [ ]:
def train_autoencoder(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def train_symmetry_net(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)

        # Generate target symmetry values (you may need to adjust this based on your specific requirements)
        target_symmetry = torch.zeros_like(outputs)
        target_symmetry[:, 0] = 0.5  # Vertical symmetry at the center
        target_symmetry[:, 1] = 0.5  # Horizontal symmetry at the center

        loss = criterion(outputs, target_symmetry)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder.to(device)
symmetry_net.to(device)

num_epochs = 50

for epoch in range(num_epochs):
    autoencoder_loss = train_autoencoder(autoencoder, train_loader, autoencoder_criterion, autoencoder_optimizer, device)
    symmetry_loss = train_symmetry_net(symmetry_net, train_loader, symmetry_criterion, symmetry_optimizer, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Autoencoder Loss: {autoencoder_loss:.4f}, Symmetry Loss: {symmetry_loss:.4f}")


In [ ]:
torch.save(autoencoder.state_dict(), 'autoencoder.pth')
torch.save(symmetry_net.state_dict(), 'symmetry_net.pth')

In [ ]:
def process_doodle(image_path, autoencoder, symmetry_net):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64))
    image_tensor = transforms.ToTensor()(image).unsqueeze(0).to(device)

    # Regularize the image
    with torch.no_grad():
        regularized = autoencoder(image_tensor)

    # Find symmetry lines
    with torch.no_grad():
        symmetry = symmetry_net(regularized)

    # Convert tensors to numpy arrays for visualization
    original = image_tensor.squeeze().cpu().numpy()
    regularized = regularized.squeeze().cpu().numpy()
    symmetry = symmetry.squeeze().cpu().numpy()

    # Visualize results
    print("Original Image:")
    cv2_imshow(original * 255)
    print("Regularized Image:")
    cv2_imshow(regularized * 255)

    # Draw symmetry lines
    result = cv2.cvtColor((regularized * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    height, width = regularized.shape
    cv2.line(result, (int(symmetry[0] * width), 0), (int(symmetry[0] * width), height), (0, 0, 255), 2)  # Vertical
    cv2.line(result, (0, int(symmetry[1] * height)), (width, int(symmetry[1] * height)), (0, 255, 0), 2)  # Horizontal

    print("Image with Symmetry Lines:")
    cv2_imshow(result)

# Load trained models
autoencoder.load_state_dict(torch.load('autoencoder.pth'))
symmetry_net.load_state_dict(torch.load('symmetry_net.pth'))
autoencoder.eval()
symmetry_net.eval()

# Process a doodle
image_path = "path/to/test/doodle.png"
process_doodle(image_path, autoencoder, symmetry_net)